# DELTA_DIST+H - Recherche des hyperparamètres

Le modèle DELTA_DIST+H est le premier que l'on va entraîner. L'objectif est d'obtenir un modèle capable d'optimiser des molécules à partir des coordonnées non optimisées des atomes. La méthode d'entraînement est la suivante : on part des coordonnées optimisées des atomes de la molécule, on leur ajoute un bruit gaussien, on calcule les distances des atomes bruités par rapport aux atomes fictifs du repère, puis on tente d'apprendre au modèle à prédire la différence entre les distances des atomes bruités et les distances des atomes optimisés.

Dans ce notebook on propose une solution (recherche par quadrillage) pour faire varier les hyperparamètres du modèle et chercher leurs meilleures valeurs pour entraîner ensuite un modèle possédant les hyperparamètres produisant les meilleurs résultats. 

#### Chemins des fichiers

In [1]:
minimal_prepared_input_loc = "../data/minimal_set_riken_v2_prepared_input.h5"
minimal_labels_loc = "../data/minimal_set_riken_v2_labels.h5"

train_prepared_input_loc = "../data/train_set_riken_v2_prepared_input.h5"
train_labels_loc = "../data/train_set_riken_v2_labels.h5"

models_loc = "../models/DELTA_DIST+H_01/models/"
logs_loc = "../models/DELTA_DIST+H_01/logs/"

## Création du RN

### Fonctions de coût et d'évaluation du modèle

#### Fonction renvoyant le masque des atomes à prendre en compte pour les calculs

L'entrée et la sortie du RN étant définies par une méthode de padding, seul un certain nombre d'entrées et de sortie est utilisé pour chaque exemple d'entraînement en fonction du nombre d'atomes de la molécule. On définit ici une fonction qui renvoie le masque des différences de distances à prendre en compte sur les données en entrée et les étiquettes.

In [2]:
import tensorflow as tf

def calcul_masque_atomes_definis(targets):
    """ On calcule le masque booléen des atomes donnés en entrée du RN en fonction du vecteur targets"""
    
    # On cherche à obtenir un masque booléen des atomes définis en entrée. Pour cela, on prend en entrée
    # les étiquettes sous la forme d'une matrice (200, 4) dont chaque ligne i est la distance de l'atome i avec
    # les atomes fictifs du repère. L'atome est indéfini ssi. la somme de la ligne est nulle. En effet,
    # un atome défini ne peut pas avoir une distance nulle avec les quatre atomes fictifs, et on veille
    # à ce que le vecteurs targets ne contienne que des valeurs nulles pour les atomes non définis.
    # On obtient donc un masque booléen de tous les atomes définis en entrée
    
    ## On somme les distances de chaque atome ##
    targets_dists_sums = tf.reduce_sum(targets, 1)
    
    ## On créé le masque des sommes différentes de zéro ##
    
    # Création des matrice de True et de False de la dimension de la matrice des sommes (nécessaires
    # pour tf.where)
    zeros = tf.cast(tf.zeros_like(targets_dists_sums),dtype=tf.bool)
    ones = tf.cast(tf.ones_like(targets_dists_sums),dtype=tf.bool)
    
    return tf.where(targets_dists_sums>0, ones, zeros)


/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Fonction de coût

In [3]:
def partial_rmse(predictions, targets):
    """ Calcule le RMSE partiel des prédictions par rapport aux valeurs attendues. Le RMSE est partiel car
    on ne le calcule que pour les sorties correspondant aux atomes donnés en entrée. En d'autres
    termes, on ne pousse pas le modèle à donner des distances nulles pour les atomes indéfinis
    en entrée"""
    

    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    
    # On masque les prédictions et les étiquettes selon le masque des atomes définis
    targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
    predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)
    
    return tf.sqrt(tf.reduce_mean(tf.squared_difference(predictions_masked, targets_masked)), name="rmse")
    

#### Fonction d'évaluation des performances (score R2)

In [4]:
def partial_r2_score(predictions, targets, inputs):
    """ Renvoie le score R2 de la prédiction (le calcul est effectué uniquement sur les résultats
    des atomes donnés en entrée) """
    
    # On met les prédictions et les cibles sous la forme d'une matrice (200, 4)
    predictions = tf.reshape(predictions, [-1, 4])
    targets = tf.reshape(targets, [-1, 4])
    
    # On calcule le masque des atomes définis selon les cibles
    defined_atoms_mask = calcul_masque_atomes_definis(targets)
    
    # On masque les prédictions et les étiquettes selon le masque des atomes définis
    targets_masked = tf.boolean_mask(targets, defined_atoms_mask)
    predictions_masked = tf.boolean_mask(predictions, defined_atoms_mask)
    
    # Calcul de l'erreur totale
    total_error = tf.reduce_sum(tf.square(tf.subtract(targets, tf.reduce_mean(targets_masked))))
                
    # Calcul de l'erreur inexpliquée
    unexplained_error = tf.reduce_sum(tf.square(tf.subtract(targets_masked, predictions_masked)))
    
    r2 = tf.subtract(1.0, tf.divide(unexplained_error, total_error), "r2_score")

#### Définition d'une fonction créant le RN

In [5]:
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import DataPreprocessing

def preprocess_fun(array):
    # print(array)
    return array.tolist()

def creer_fc3_1k_4x15k_800(do=0.99, stddev_init=0.001, learning_rate=0.05):
    """ Fonction créant un réseau de neurones de type fully connected, ayant une couche d'entrée de 1000
    neurones, quatre couches cachées de 15000 neurones et une sortie de 800 neurones (en plus d'une couche 
    d'un neurone de calcul du coût)
    Inputs 
    dropout : proba que chaque neurone ne soit pas désactivé
    stddev_init : sigma de l'initialisation des poids
    learning_rate : taux d'apprentissage
    """

    # On créé l'initialisateur de tenseur avec une loi normale tronquée. sigma = stddev_init, et les 
    # valeurs à plus de 2sigma sont re-tirées
    winit = tfl.initializations.truncated_normal(stddev=stddev_init, dtype=tf.float32, seed=None)

    preprocess = DataPreprocessing(name="preprocessor")
    preprocess.add_custom_preprocessing(preprocess_fun)
    
    # On créé l'input du RN
    network = input_data(shape=[None, 1000], name='input', data_preprocessing=preprocess)

    
    # On créé les quatre couches cachées
    for i in range(1):
        network = fully_connected(network, 15000, activation='relu', name='fc'+str(i), weights_init=winit)
        
    # On détruit des neurones aléatoirement avec une la probabilité donnée en entrée
    network = dropout(network, do)
    

    # On ajoute la couche de sortie du réseau
    # Fonction d'activation prelu
    # Initilisée avec la loi normale tronquée
    network = fully_connected(network, 800, activation='prelu', name='outlayer', weights_init=winit)
    # Couche d'évaluation du modèle. Utilisation d'une descente stochastique Adam
    # Learning rate = 0.05
    # Loss = fonction définie rmse
    network = regression(network, optimizer='adam', learning_rate=0.05,
        loss=partial_rmse, metric=partial_r2_score, name='target')
    
    tf.trainable_variables()
    
    print("Fin création RN")
    
    return network

Instructions for updating:
Use the retry module or similar alternatives.


## Préparation des données

#### Fonction renvoyant deux sous-ensembles du jeu d'entrainement : un ensemble d'exemples et les cibles correspondantes

In [6]:
def get_fold(train_set, targets, reduce_train_fold_proportion):
    """ Permet d'obtenir deux sous-ensembles du jeu d'entraînement afin de ne pas travailler sur le jeu
    d'entraînement total pour la recherche par quadrillage et donc de gagner du temps d'exécution. L'idée
    et que si un ensemble d'hyper-paramètres produit des meilleurs résultats que les autres ensembles
    d'hyper-paramètres sur l'ensemble du jeu d'entraînement, alors on suppose que ce sera également 
    le cas sur une partie des données. """
            
    size = int(len(train_set["inputs"]) * reduce_train_fold_proportion)
    print(size)
    return (np.array(train_set["inputs"][:size].flat), np.array(targets["targets"][:size]))
    
    

## Entraînement des modèles

#### Fonction d'entraînement de modèles pour recherche des meilleurs hyperparamètres

In [7]:
import h5py
import tflearn as tfl
import time
from scipy import sparse
import numpy as np


def train(input_path, targets_path, fold_prop):
    
    inputs_h5 = h5py.File(input_path, 'r')
    targets_h5 = h5py.File(targets_path, 'r')
    
    dropout_vals = [0.99, 0.95]
    stddev_init_vals = [0.001, 0.003]
    learning_rate_vals = [0.05, 0.1]
    
    input_X, labels_y = get_fold(inputs_h5, targets_h5, fold_prop)
    
    input_X = inputs_h5["inputs"][:500]
    print(input_X)
    targets_y = targets_h5["targets"][:500] 
 
    total_start_time = time.time()
    
    # On itère sur toutes les combinaisons d'hyperparamètres
    for dropout in dropout_vals:
        for stddev_init in stddev_init_vals:
            for learning_rate in learning_rate_vals:
                
                start_time = time.time()
                
                nom_modele = "DELTA_DIST+H_01_hyperparam_do-"+str(dropout)+"_stdinit_"+str(stddev_init)+"_lr_"+str(learning_rate)
                
                print("Training model "+nom_modele)
                
                
                # On créé le réseau avec les hyperparamètres courants
                network = creer_fc3_1k_4x15k_800(do=dropout, learning_rate=learning_rate, 
                                                 stddev_init=stddev_init)
                
                # On créé le modèle
                model = tfl.DNN(network, tensorboard_verbose=3,
                tensorboard_dir=logs_loc)
            
                # Entraînement
                model.fit(X_inputs=input_X,Y_targets=labels_y, batch_size=100,
                          shuffle = True, snapshot_step=100, validation_set=0.1,
                          show_metric=True, run_id="hyperparametres_"+nom_modele , n_epoch=5)

                model.save(models_loc+"test_" + nom_modele)
                
                print("Temps d'entraînement du modèle : ")
                print("--- %s seconds ---" % (time.time() - total_start_time))

    
    
    print("Temps total :")
    print("--- %s seconds ---" % (time.time() - total_start_time))



In [8]:
train(minimal_prepared_input_loc, minimal_labels_loc, 0.01)

Training Step: 24  | total loss: 19.37223 | time: 3.662s
| Adam | epoch: 005 | loss: 19.37223 -- iter: 400/450
Training Step: 25  | total loss: 19.30609 | time: 5.581s
| Adam | epoch: 005 | loss: 19.30609 | val_loss: 20.27363 -- iter: 450/450
--
INFO:tensorflow:/home/etudiant/Documents/Cours/TER/Notebooks/models/DELTA_DIST+H_01/models/test_DELTA_DIST+H_01_hyperparam_do-0.99_stdinit_0.001_lr_0.05 is not in all_model_checkpoint_paths. Manually adding it.
Temps d'entraînement du modèle : 
--- 30.669343948364258 seconds ---
Training model DELTA_DIST+H_01_hyperparam_do-0.99_stdinit_0.001_lr_0.1
Fin création RN


IndexError: list index out of range

In [10]:
import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [11]:
 print(np.array(X))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
print(X[:2])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
 x = np.array([[1, 2, 3], [4, 5, 6]])
print(x)

[[1 2 3]
 [4 5 6]]


In [1]:
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()

/home/etudiant/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


100.0% 170500096 / 170498071


('Succesfully downloaded', 'cifar-10-python.tar.gz', 170498071, 'bytes.')
File Extracted in Current Directory


In [3]:
print(X[:1])

[[[[0.23137255 0.24313725 0.24705882]
   [0.16862745 0.18039216 0.17647059]
   [0.19607843 0.18823529 0.16862745]
   ...
   [0.61960784 0.51764706 0.42352941]
   [0.59607843 0.49019608 0.4       ]
   [0.58039216 0.48627451 0.40392157]]

  [[0.0627451  0.07843137 0.07843137]
   [0.         0.         0.        ]
   [0.07058824 0.03137255 0.        ]
   ...
   [0.48235294 0.34509804 0.21568627]
   [0.46666667 0.3254902  0.19607843]
   [0.47843137 0.34117647 0.22352941]]

  [[0.09803922 0.09411765 0.08235294]
   [0.0627451  0.02745098 0.        ]
   [0.19215686 0.10588235 0.03137255]
   ...
   [0.4627451  0.32941176 0.19607843]
   [0.47058824 0.32941176 0.19607843]
   [0.42745098 0.28627451 0.16470588]]

  ...

  [[0.81568627 0.66666667 0.37647059]
   [0.78823529 0.6        0.13333333]
   [0.77647059 0.63137255 0.10196078]
   ...
   [0.62745098 0.52156863 0.2745098 ]
   [0.21960784 0.12156863 0.02745098]
   [0.20784314 0.13333333 0.07843137]]

  [[0.70588235 0.54509804 0.37647059]
   [0.6